# using TorchText for batched training  
Modifiy from  https://torchtutorialstaging.z5.web.core.windows.net/beginner/text_sentiment_ngrams_tutorial.html   
and then add the convolution layer according to https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-conv1d-for-text-classification#4


In [ ]:
# loading data from worksapce

from azureml.core import Workspace, Dataset
subscription_id = '26ca7667-25cd-40ca-9ab0-aa0cb6fbdb97'
resource_group = 'SBN_Analytics_Sandbox'
workspace_name = 'azureML'
workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='sample200').to_pandas_dataframe()
testset = Dataset.get_by_name(workspace, name='test').to_pandas_dataframe()

In [1]:
# load data from local filesystem

import pandas as pd
from sklearn.utils import resample

#df = pd.read_csv('../../data/fullPOItemWithUnspsc/train.csv')
dataset = pd.read_csv('../../../data/fullPOItemWithUnspsc/train_50.csv')
testset = pd.read_csv('../../../data/fullPOItemWithUnspsc/test.csv')
# cnt = pd.read_csv('../../data/fullPOItemWithUnspsc/full_class_cnt.csv')


In [ ]:


n_threshold = 300   # merge those classes have n_threshold or less record as others
n_traget = 500     # re-sample each class to n_traget record

dataset = pd.DataFrame()
for idx, row in cnt[cnt.COUNT>n_threshold].iterrows():
    #print( "UNSPSC:", row.CODE, "count:", row.COUNT)
    df_temp = resample( df[df.CODE == row.CODE], n_samples = n_traget, random_state=51)
    dataset = pd.concat([dataset, df_temp], ignore_index=True)

others  = cnt[cnt.COUNT <= n_threshold].CODE.tolist()
df_temp = resample( df[ df.CODE.isin( others ) ], n_samples = n_traget, random_state=51)
df_temp.CODE = 0
dataset = pd.concat([dataset, df_temp], ignore_index=True)

testset.CODE = testset.CODE.apply( lambda x: 0 if x in others else x)

print(dataset.shape)


In [2]:

# dataset = dataset.drop_duplicates()

from sklearn import preprocessing

labelEncoder = preprocessing.LabelEncoder()
dataset["LABEL"] = labelEncoder.fit_transform( dataset.CODE )
testset["LABEL"] = labelEncoder.transform( testset.CODE )

dataset.head


<bound method NDFrame.head of                                                DESCRIPTION      CODE  LABEL
0        Wealth Management Double Sided Team BC with Ti...  14111604   1205
1        1 Pack General Flat Offset Business Card for A...  14111604   1205
2        BC Standard Business Card 250 Box David Segura...  14111604   1205
3        220201 Business Card Flat PDF F568084 Takahito...  14111604   1205
4        MS Canada Limited Business Card 500 David J Ha...  14111604   1205
...                                                    ...       ...    ...
1013195           Material for Scenario 10 Partial receipt  40101509   7227
1013196           Material for Scenario 10 Partial receipt  40101509   7227
1013197           Material for Scenario 10 Partial receipt  40101509   7227
1013198           Material for Scenario 10 Partial receipt  40101509   7227
1013199           Material for Scenario 10 Partial receipt  40101509   7227

[1013200 rows x 3 columns]>

In [3]:

import torch
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


tokenizer = get_tokenizer('basic_english')
train_iter = dataset.itertuples()

def yield_tokens(data_iter):
    for row in data_iter:
        yield tokenizer( getattr(row, "DESCRIPTION") )

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))

print( "size of vocabulary is",len(vocab))

size of vocabulary is 604206


In [4]:

text_pipeline('test 200mA usb 苹果')


[67, 21111, 220, 0]

In [92]:

max_tokens = 50

from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for ( idx , row) in batch:
        # print(row)
        label_list.append(getattr(row, 'LABEL'))

        tokens = text_pipeline(getattr(row, 'DESCRIPTION'))
 
        # if max_tokens defined, cutting or padding tokens to max_tokens
        if max_tokens > 0:
            tokens = tokens+([0]* (max_tokens-len(tokens))) if len(tokens)<max_tokens else tokens[:max_tokens]

        processed_text = torch.tensor(tokens, dtype=torch.int64)
 
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)



cpu


In [93]:


from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embed_dim = embed_dim
        #self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1 = nn.Conv1d(embed_dim, 1024, kernel_size=7, padding="same")
#        self.fc1 = nn.Linear(embed_dim, 2000)
#        self.layer1 = nn.Sequential(self.fc1, nn.ReLU(), nn.Dropout(p=0.1))
#        self.fc2 = nn.Linear(2000, 2000)
#        self.layer2 = nn.Sequential(self.fc2, nn.ReLU(), nn.Dropout(p=0.1))
        self.fc3 = nn.Linear(1024, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.conv1.weight.data.uniform_(-initrange, initrange)
        self.conv1.bias.data.zero_()
#        self.fc1.weight.data.uniform_(-initrange, initrange)
#        self.fc1.bias.data.zero_()
#        self.fc2.weight.data.uniform_(-initrange, initrange)
#        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
#        out = self.embedding(text, offsets)
        out = self.embedding(text.reshape( len(offsets),  max_tokens ))
        #out = out.mean(dim=1)
        out = out.reshape(len(out), self.embed_dim, max_tokens)
        out = nn.functional.relu(self.conv1(out))
        out, _ = out.max(dim=-1)

#        out = self.layer1(out)
#        out = self.layer2(out)
        return  self.fc3(out)


#train_iter = AG_NEWS(split='train')
num_class = len(labelEncoder.classes_)
vocab_size = len(vocab)
#emsize = 64
emsize = 2048
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

#model.train()
#t = torch.tensor([0]*100, dtype=torch.int64)
#model( t ,[0,50])


In [94]:



import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [ ]:



from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 #64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
# train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(dataset.iterrows())
#test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
#test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
#                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    # not sure if empty cache helps to solve the OOM error
    # torch.cuda.empty_cache()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

In [ ]:


test_dataset = to_map_style_dataset(testset.iterrows())
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))




test accuracy    0.540


In [ ]:

print( evaluate(valid_dataloader) )


0.6545270484024043
